In [1]:
import pandas as pd
import configparser
from sqlalchemy import create_engine, text

In [2]:
config = configparser.ConfigParser()
config.read("querycrew.ini")
#Read the ini file to login into the database
db_name = config["mysql"]["database"]
db_host = config["mysql"]["host"]
db_user = config["mysql"]["user"]
db_password = config["mysql"]["password"]

#Connect
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")

conn = engine.connect()


In [3]:
query =text(( """
SELECT 
    vehicles.vehicle_id, 
    vehicles.vin, 
    vehicles.make, 
    vehicles.model, 
    vehicles.year, 
    Vehicle.model_year, 
    Vehicle.base_msrp
FROM 
    vehicles
RIGHT JOIN 
    Vehicle 
ON 
    vehicles.vin = Vehicle.vin;
"""))
result = conn.execute(query)
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df.head()

,vehicle_id,vin,make,model,year,model_year,base_msrp
0,None,None,None,None,None,2020,30000
1,None,None,None,None,None,2019,25000
2,None,None,None,None,None,2021,35000
3,None,None,None,None,None,2018,20000
4,None,None,None,None,None,2022,40000


In [4]:
query =text(( """
SELECT 
    vehicles.vehicle_id, 
    vehicles.vin, 
    vehicles.make, 
    vehicles.model, 
    vehicles.year, 
    Vehicle.model_year, 
    Vehicle.base_msrp
FROM 
    vehicles
LEFT JOIN 
    Vehicle 
ON 
    vehicles.vin = Vehicle.vin;
"""))
result = conn.execute(query)
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df.head()

,vehicle_id,vin,make,model,year,model_year,base_msrp
0,1,1C4JJXP66P,JEEP,WRANGLER,2023,None,None
1,2,1G1FX6S08K,CHEVROLET,BOLT EV,2019,None,None
2,3,WBY1Z2C58F,BMW,I3,2015,None,None
3,4,5YJ3E1EBXK,TESLA,MODEL 3,2019,None,None
4,5,5YJSA1V24F,TESLA,MODEL S,2015,None,None


In [5]:
query =text(( """
SELECT 
    vehicles.vehicle_id, 
    vehicles.vin, 
    vehicles.make, 
    vehicles.model, 
    vehicles.year, 
    Vehicle.model_year, 
    Vehicle.base_msrp
FROM 
    vehicles
LEFT JOIN 
    Vehicle 
ON 
    vehicles.vin = Vehicle.vin
UNION
SELECT 
    vehicles.vehicle_id, 
    vehicles.vin, 
    vehicles.make, 
    vehicles.model, 
    vehicles.year, 
    Vehicle.model_year, 
    Vehicle.base_msrp
FROM 
    vehicles
RIGHT JOIN 
    Vehicle 
ON 
    vehicles.vin = Vehicle.vin
WHERE 
    vehicles.vin IS NULL;
"""))
result = conn.execute(query)
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df.head()

,vehicle_id,vin,make,model,year,model_year,base_msrp
0,1.0,1C4JJXP66P,JEEP,WRANGLER,2023.0,NaN,NaN
1,2.0,1G1FX6S08K,CHEVROLET,BOLT EV,2019.0,NaN,NaN
2,3.0,WBY1Z2C58F,BMW,I3,2015.0,NaN,NaN
3,4.0,5YJ3E1EBXK,TESLA,MODEL 3,2019.0,NaN,NaN
4,5.0,5YJSA1V24F,TESLA,MODEL S,2015.0,NaN,NaN
